In [14]:
import pyodbc
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings("ignore")
# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = '10.0.1.40' 
database = 'Ntier_EPIC' 
username = 'Sisenseuser' 
password = '!w@ntd@t@#123'  
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [15]:
query1=("""Select [Patient_ID],[Service_Date_From],[Procedure_Code],[Primary_Diagnosis_Code]
                            from [vwGenSvcInfo] where Primary_Diagnosis_Code
                            between 'E08' and 'E13' and Patient_ID='103257';""")
actual_data= pd.read_sql(query1, cnxn)

actual_data["Service_Date_From"]=pd.to_datetime(actual_data["Service_Date_From"])

#actual_data.drop_duplicates(['Service_Date_To','Procedure_Code'],inplace=True)
actual_data

,Patient_ID,Service_Date_From,Procedure_Code,Primary_Diagnosis_Code
0,103257,2021-07-27,36416,E11.40
1,103257,2021-03-19,36416,E11.40
2,103257,2021-04-14,36416,E11.40
3,103257,2021-05-14,36416,E11.40
4,103257,2021-03-19,82962,E11.40
5,103257,2021-04-14,82962,E11.40
6,103257,2021-07-27,82962,E11.40
7,103257,2021-05-14,83036,E11.40
8,103257,2021-03-29,99072,E11.40
9,103257,2020-05-28,2022F,E11.65


In [16]:
def getderiveddata(actual_data,app_date):
    import datetime
    import datetime 
    from datetime import datetime, time, date
    #app_date="2022-07-15"
    formatter = '%Y-%m-%d'
    app_date = datetime.strptime(app_date, formatter)
    actual_data["new_date"] = app_date
    actual_data["Service_Date_From"]=pd.to_datetime(actual_data["Service_Date_From"])
    #actual_data["Service_Date_To"]=pd.to_datetime(actual_data["Service_Date_To"])
    
    actual_data["Service_Date_max"] = actual_data.groupby("Patient_ID")["Service_Date_From"].transform(max)
    actual_data["new_date"]=pd.to_datetime(actual_data["new_date"],errors='coerce')
    #actual_data["last_visit_day"] = (actual_data["new_date"] - actual_data["Service_Date_max"])
    #actual_data["last_visit_day"] = actual_data["last_visit_day"].apply(lambda x: int(str(x).split(" ")[0]))
    #actual_data["Patient_Practice"] = actual_data.loc[:, "last_visit_day"].apply(lambda x: "Adhoc" if x >= 90 else "Regular")
    actual_data["Diff_in_days"] = (actual_data["new_date"] - actual_data["Service_Date_max"])
    actual_data["Diff_in_days"] = actual_data["Diff_in_days"].apply(lambda x: int(str(x).split(" ")[0]))
    return actual_data

In [17]:
getderiveddata(actual_data,'2022-01-30') 

,Patient_ID,Service_Date_From,Procedure_Code,Primary_Diagnosis_Code,new_date,Service_Date_max,Diff_in_days
0,103257,2021-07-27,36416,E11.40,2022-01-30,2021-07-27,187
1,103257,2021-03-19,36416,E11.40,2022-01-30,2021-07-27,187
2,103257,2021-04-14,36416,E11.40,2022-01-30,2021-07-27,187
3,103257,2021-05-14,36416,E11.40,2022-01-30,2021-07-27,187
4,103257,2021-03-19,82962,E11.40,2022-01-30,2021-07-27,187
5,103257,2021-04-14,82962,E11.40,2022-01-30,2021-07-27,187
6,103257,2021-07-27,82962,E11.40,2022-01-30,2021-07-27,187
7,103257,2021-05-14,83036,E11.40,2022-01-30,2021-07-27,187
8,103257,2021-03-29,99072,E11.40,2022-01-30,2021-07-27,187
9,103257,2020-05-28,2022F,E11.65,2022-01-30,2021-07-27,187


In [18]:
rules_data=pd.read_csv("E:/OneDrive - Quadratic Insights Pvt Ltd/Desktop/Epic_Rules.csv")
rules_data

,Diabetic_Patient,Diagnosis_Code,Procedures_Description,Procedure_Code,Freq_Threshold_In_Days
0,Diabetic,E08-E13,HBA1C,83036,90
1,Diabetic,E08-E13,Urinalysis,81002,180
2,Diabetic,E08-E13,Microalbumin w/ Creatinine,82044,180
3,Diabetic,E08-E13,Microalbumin w/ Creatinine,82570,180
4,Diabetic,E08-E13,CMP (comprehensive metabolic panel),80053,90
5,Diabetic,E08-E13,Lipid,80061,120
6,Diabetic,E08-E13,TSH,84443,180
7,Diabetic,E08-E13,Foot exam,2028F,365
8,Diabetic,E08-E13,RETINAL EYE EXAM,92250,365
9,Diabetic,E08-E13,DILATED EYE EXAMINATION PERFORMED,2022F,365


In [25]:
def recommendation(one_patient_record, rules_data):
    recommendation_procedure_list = {}
    # aic_procedure_code = '3046f'
#     aic_procedure_code = '83036'
#     aic_procedure_code_value=list(one_patient_record[one_patient_record['Procedure_Code']=='83036']['RESULT_VALUE'])[0]
    unique_procedure_code = rules_data["Procedure_Code"].value_counts().keys().tolist()

    for row in range(len(one_patient_record)):
        procedure_code_list = []
        diff_in_days_list = []
                # result_value = []

        procedure_code_list.append(one_patient_record.loc[row, "Procedure_Code"])
        diff_in_days_list.append(one_patient_record.loc[row, "Diff_in_days"])
                #result_value.append(one_patient_record.loc[row, "RESULT_VALUE"])

        for record in range(len(procedure_code_list)):
            procedure_code = procedure_code_list[record]
            diff_in_days = diff_in_days_list[record]

            if procedure_code in unique_procedure_code:
                res = rules_data.loc[rules_data["Procedure_Code"] == procedure_code]
                rules_procedure_code = res["Procedure_Code"].values[0]
                rules_diff_in_days = res["Freq_Threshold_In_Days"].values[0]

                if rules_procedure_code == procedure_code and int(rules_diff_in_days) <= (diff_in_days):
                    recommendation_procedure_list[procedure_code] = "1"
                else:
                    recommendation_procedure_list[procedure_code] = "0"
                    
#                 if procedure_code == aic_procedure_code:
#                     if (rules_diff_in_days) >= (diff_in_days) and aic_procedure_code_value < "7":
#                         recommendation_procedure_list[procedure_code] = "1"
#                     elif (rules_diff_in_days) >= (diff_in_days) and aic_procedure_code_value < "8":
#                         recommendation_procedure_list[procedure_code] = "1"
#                     elif (rules_diff_in_days) >= (diff_in_days) and aic_procedure_code_value < "9":
#                         recommendation_procedure_list[procedure_code] = "1"
#                     else:
#                         recommendation_procedure_list[procedure_code] = "0"
                    
            else:
                for procedure in unique_procedure_code:
                    if procedure not in list(recommendation_procedure_list.keys()):
                        recommendation_procedure_list[procedure] = "1"


    return recommendation_procedure_list


In [26]:
recommendation(actual_data, rules_data)

{'83036': '1',
 '81002': '1',
 '82044': '1',
 '82570': '1',
 '80053': '1',
 '80061': '1',
 '84443': '1',
 '2028F': '0',
 '92250': '0',
 '2022F': '0',
 '2026F': '0',
 '93922': '1'}

# Patient_data

In [106]:
import datetime
import logging
import sys
from datetime import datetime, time, date
import warnings
import pandas as pd
import numpy as np
from django.db import connection
warnings.filterwarnings("ignore")

# import struct

formatter = '%Y-%m-%d'
logger = logging.getLogger(__name__)


class PatientData:
    def __init__(self):
        self.logger = logging.getLogger(__name__)
        self.sName = "Recommend Procedure Service"
        self.status = 0
        extra = {
            "cls_name": self.__class__.__name__,
        }
        self.logger = logging.LoggerAdapter(self.logger, extra)

    def get_patient_service_history(self, patient_id):
        try:

            cursor1 = connection.cursor()
            cursor1.execute("""Select [Patient_ID],[Service_Date_From],[Procedure_Code],
                            [Primary_Diagnosis_Code]
							from Ntier_EPIC.dbo.[vwGenSvcInfo] where Primary_Diagnosis_Code
                            between 'E08' and 'E13' and Patient_ID = %s """, (patient_id,))

            result1 = pd.DataFrame([list(elem) for elem in cursor1.fetchall()])

            if result1.empty:
                result_dict1 = {"response_key": "-1"}
            else:
                self.logger.info("result1 == " + str(result1))
                result1.columns = ['Patient_ID', 'Service_Date_From', 'Procedure_Code', 'Primary_Diagnosis_Code']
                result1.sort_values(by='Service_Date_From', inplace=True, ascending=False)
                result_dict1 = {"response_key": result1}

        except Exception as e:
            self.logger.exception(str(e))
            print("Exception  ", str(e))
            result_dict1 = {"response_key": "-1"}

        return result_dict1

    def get_patient_lab_result(self, patient_id):
        try:

            cursor2 = connection.cursor()

            cursor2.execute("""Select T3.Patient_ID,T1.IMREDEM_CODE,T1.RESULT_ABNORMAL, T1.RESULT_NAME, 
                                    T1.RESULT_VALUE, T1.RESULT_UNITS,T1.RESULT_NORMAL_RANGE,CONVERT(VARCHAR(10),T1.RESULT_COMPLETIONDATE, 111) AS RESULT_COMPLETIONDATE
									--CAST(T1.RESULT_COMPLETIONDATE as nvarchar(100)) RESULT_COMPLETIONDATE 
                                    FROM EMR.[HPSITE].LAB_RESULT AS T1 
                                    LEFT JOIN EMR.[HPSITE].[DEMOGRAPHICS] AS T2 ON T1.IMREDEM_CODE=T2.IMREDEM_CODE
                                    LEFT JOIN Ntier_EPIC.dbo.vwGenPatInfo T3 ON T2.DEM_EXTERNALID=T3.Patient_Number
                                    where (T1.RESULT_VALUE IS NOT NULL) and (T1.RESULT_NORMAL_RANGE IS NOT NULL) and T3.Patient_ID = %s """, (patient_id,))

            result2 = pd.DataFrame([list(elem) for elem in cursor2.fetchall()])

            if result2.empty:
                result_dict2 = {"response_key": "-1"}
            else:
                result2.columns = ['Patient_ID', 'IMREDEM_CODE', 'RESULT_ABNORMAL', 'RESULT_NAME', 'RESULT_VALUE',
                                   'RESULT_UNITS', 'RESULT_NORMAL_RANGE', 'RESULT_COMPLETIONDATE']

                result2.sort_values(by='RESULT_COMPLETIONDATE', inplace=True, ascending=False)
                self.logger.info("result2 == " + str(result2))

                result_dict2 = {"response_key": result2}

        except Exception as e:
            self.logger.exception(str(e))
            result_dict2 = {"response_key": "-1"}

        return result_dict2

    def get_min_max_daterange(self,Patient_data,Labresult_data):
        try:
            Labresult_data["RESULT_COMPLETIONDATE"] = pd.to_datetime(Labresult_data["RESULT_COMPLETIONDATE"])
            Patient_data['Service_Date_From_trial'] = pd.to_datetime(Patient_data['Service_Date_From'])
            # date= Patient_data['Service_Date_From_trial']
            Patient_data['Min'] = Patient_data['Service_Date_From_trial']
            remove = lambda x: Patient_data[x].duplicated(keep='last')
            Patient_data.loc[remove('Service_Date_From_trial'), 'Service_Date_From_trial'] = np.nan
            p = Patient_data['Service_Date_From_trial'][:-1]
            p.index = p.index + 1
            Patient_data['Max'] = pd.concat([pd.Series(['']), p])
            Patient_data['Min'] = pd.to_datetime(Patient_data['Min'])
            Patient_data['Max'] = pd.to_datetime(Patient_data['Max'])
            m = Patient_data['Max'].duplicated()
            Patient_data['Max'] = Patient_data['Max'].mask(m).ffill()
            Patient_data.drop('Service_Date_From_trial', axis=1, inplace=True)
            Patient_data["Service_Date_From"] = pd.to_datetime(Patient_data["Service_Date_From"])
            Patient_data["Service_Date_From"] = Patient_data["Service_Date_From"].astype(str)
            min_date, max_date = Patient_data["Service_Date_From"].min(), Patient_data["Service_Date_From"].max()
            lists_of_labcomp_data = Labresult_data["RESULT_COMPLETIONDATE"].unique()
            lists_of_labcomp_data = [pd.to_datetime(str(i)) for i in lists_of_labcomp_data]
            lists_of_labcomp_data.sort()
            date_range = pd.date_range(min_date, max_date)
            date_range = [pd.to_datetime(str(i)) for i in date_range]  # parser.parse(str(i)).strftime('%Y-%m-%d')
            results = list(set(date_range).intersection(set(lists_of_labcomp_data)))
            results.sort(reverse=True)
            result_list = results
            resultant_df = pd.DataFrame()
            for i in range(len(result_list)):
                res = Labresult_data.loc[Labresult_data["RESULT_COMPLETIONDATE"] == result_list[i]]
                resultant_df = resultant_df.append(res)
            return Patient_data, Labresult_data, resultant_df

        except Exception as e:
            self.logger.exception(str(e))
            exception_type, exception_object, exception_traceback = sys.exc_info()
            filename = exception_traceback.tb_frame.f_code.co_filename
            line_number = exception_traceback.tb_lineno

            print("Exception type: ", exception_type)
            print("File name: ", filename)
            print("Line number: ", line_number)

    def get_final_df_with_lab_results(self,Patient_data, resultant_df):
        try:
            df_li = []
            today = date.today()
            str(today)
            Patient_data['LabResult'] = None
            lab_result_dates = list(set(resultant_df['RESULT_COMPLETIONDATE'].values.tolist()))
            lab_result_dates_new = [pd.to_datetime(l) for l in lab_result_dates]
            lab_result_dates_new.sort(reverse=True)
            Lab_result_list1 = lab_result_dates_new

            for index, rows in Patient_data.iterrows():
                status = 0
                if rows['Procedure_Code'] == "83036":
                    min_date = rows['Min']
                    max_date = rows['Max']
                    if str(max_date) == "NaT":
                        max_date = min_date
                        now = date.today()
                        rows['Max'] = now

                    for each_result_date in Lab_result_list1:
                        each_result_date = pd.to_datetime(each_result_date)
                        if each_result_date >= min_date and each_result_date <= max_date:
                            rows['LabResult'] = each_result_date
                            df_li.append(rows)
                            df_li.append(rows.copy())
                            status = 1
                if status == 0:
                    df_li.append(rows)

            Patient_data_new = pd.DataFrame(df_li)
            Patient_data_new['RESULT_COMPLETIONDATE'] = Patient_data_new['LabResult'].apply(
                lambda x: x.strftime("%Y-%m-%d") if str(x) != "NaT" else "NA")
            resultant_df['RESULT_COMPLETIONDATE'] = resultant_df['RESULT_COMPLETIONDATE'].apply(
                lambda x: x.strftime("%Y-%m-%d") if str(x) != "NaT" else "NA")
            patient_df_lab_record_join = Patient_data_new.merge(resultant_df, on="RESULT_COMPLETIONDATE", how="left")
            return patient_df_lab_record_join, resultant_df

        except Exception as e:
            self.logger.exception(str(e))
            exception_type, exception_object, exception_traceback = sys.exc_info()
            filename = exception_traceback.tb_frame.f_code.co_filename
            line_number = exception_traceback.tb_lineno

            print("Exception type: ", exception_type)
            print("File name: ", filename)
            print("Line number: ", line_number)

    def get_patient_derived_data(self,patient_df_lab_record_join, app_date):
        try:
            app_date = datetime.strptime(app_date, formatter)
            patient_df_lab_record_join.reset_index(drop=True)
            patient_df_lab_record_join["Next_Appt_date"] = app_date
            patient_df_lab_record_join["Service_Date_From"] = pd.to_datetime(patient_df_lab_record_join["Service_Date_From"])
            patient_df_lab_record_join["Service_Date_max"] = patient_df_lab_record_join.groupby("Patient_ID_x")["Service_Date_From"].transform(max)
            patient_df_lab_record_join["Diff_in_days"] = (patient_df_lab_record_join["Next_Appt_date"] - patient_df_lab_record_join["Service_Date_max"])
            patient_df_lab_record_join["Diff_in_days"] = patient_df_lab_record_join["Diff_in_days"].apply(
                lambda x: int(str(x).split(" ")[0]))
            patient_df_lab_record_join["Patient_Practice"] = patient_df_lab_record_join.loc[:, "Diff_in_days"].apply(
                lambda x: "Adhoc" if x >= 90 else "Regular")
            patient_df_lab_record_join = patient_df_lab_record_join[
                ['Patient_ID_x', 'Service_Date_From', 'Procedure_Code', 'Primary_Diagnosis_Code', 'Next_Appt_date',
                 'Service_Date_max', 'Diff_in_days', 'Patient_Practice', 'RESULT_COMPLETIONDATE', 'RESULT_ABNORMAL',
                 'RESULT_NAME', 'RESULT_VALUE', 'RESULT_NORMAL_RANGE']]
            return patient_df_lab_record_join

        except Exception as e:
            self.logger.exception(str(e))
            exception_type, exception_object, exception_traceback = sys.exc_info()
            filename = exception_traceback.tb_frame.f_code.co_filename
            line_number = exception_traceback.tb_lineno

            print("Exception type: ", exception_type)
            print("File name: ", filename)
            print("Line number: ", line_number)

    def recommendation(self,one_patient_record, rules_data):
        try:
            recommendation_procedure_list = {}
            # aic_procedure_code = '3046f'
            unique_procedure_code = rules_data["Procedure_Code"].value_counts().keys().tolist()

            for row in range(len(one_patient_record)):
                procedure_code_list = []
                diff_in_days_list = []
                # result_value = []

                procedure_code_list.append(one_patient_record.loc[row, "Procedure_Code"])
                diff_in_days_list.append(one_patient_record.loc[row, "Diff_in_days"])
                # result_value.append(one_patient_record.loc[row, "RESULT_VALUE"])

                for record in range(len(procedure_code_list)):
                    procedure_code = procedure_code_list[record]
                    diff_in_days = diff_in_days_list[record]

                    if procedure_code in unique_procedure_code:
                        res = rules_data.loc[rules_data["Procedure_Code"] == procedure_code]
                        rules_procedure_code = res["Procedure_Code"].values[0]
                        rules_diff_in_days = res["Freq_Threshold_In_Days"].values[0]

                        if rules_procedure_code == procedure_code and int(rules_diff_in_days) <= (diff_in_days):
                            recommendation_procedure_list[procedure_code] = "1"
                        else:
                            recommendation_procedure_list[procedure_code] = "0"

                    else:
                        for procedure in unique_procedure_code:
                            if procedure not in list(recommendation_procedure_list.keys()):
                                recommendation_procedure_list[procedure] = "1"

            return recommendation_procedure_list
        except Exception as e:
            self.logger.exception(str(e))
            exception_type, exception_object, exception_traceback = sys.exc_info()
            filename = exception_traceback.tb_frame.f_code.co_filename
            line_number = exception_traceback.tb_lineno

            print("Exception type: ", exception_type)
            print("File name: ", filename)
            print("Line number: ", line_number)

    def allproceducecode(self, rules_data):
        try:
            rec_result = dict()
            Standard_procedure_List = rules_data["Procedure_Code"].value_counts().keys().tolist()
            for item in Standard_procedure_List:
                rec_result[item] = "1"
            # return {str(Patient_ID): rec_result}
            return rec_result

        except Exception as e:
            self.logger.exception(str(e))
            exception_type, exception_object, exception_traceback = sys.exc_info()
            filename = exception_traceback.tb_frame.f_code.co_filename
            line_number = exception_traceback.tb_lineno

            print("Exception type: ", exception_type)
            print("File name: ", filename)
            print("Line number: ", line_number)






    #         if Patient_data.loc[0]["RESULT_ABNORMAL"] == 1:
    #             Patient_data["Reccomend_Code"] = Patient_data["Procedure_Code"]
    #
    #         elif (Patient_data.loc[0]["Diff_In_days"] < 90) and (Patient_data.loc[0]["RESULT_VALUE"] < 7):
    #             Patient_data["Reccomend_Code"] = Patient_data["Procedure_Code"]
    #
    #         elif (Patient_data.loc[0]["Diff_In_days"] < 60) and (Patient_data.loc[0]["RESULT_VALUE"] < 8):
    #             Patient_data["Reccomend_Code"] = Patient_data["Procedure_Code"]
    #
    #
    #         elif (Patient_data.loc[0]["Diff_In_days"] < 30) and (Patient_data.loc[0]["RESULT_VALUE"] < 9):
    #             Patient_data["Reccomend_Code"] = Patient_data["Procedure_Code"]
    #
    #         else:
    #             Patient_data["Reccomend_Code"] = 0
    #
    #         return Patient_data






# Views

In [ ]:
import logging
import os
import time
from configparser import ConfigParser

import pandas as pd
from django.conf import settings
from django.http import HttpResponse
from rest_framework.response import Response
from rest_framework.views import APIView

from .InputRequestValidation import InputRequestValidation
from .PatientData import PatientData
from .ResponseJson import ResponseJson
from .errorcode import ErrorCodes

config = ConfigParser()
config.read("config/config.ini")

log_dir = settings.LOG_DIR
rules_data = pd.read_csv(config["path"]["rules_data_path"])


def log_file(self):
    try:
        file_name = "RuleEngine" + ".log"
        f = open(os.path.join(log_dir, file_name), "r")
        file_contents = f.read()
        f.close()
        return HttpResponse(file_contents, content_type="text/plain")
    except Exception as e:
        return HttpResponse(str(e), content_type="text/plain")


class RecommendProcedure(APIView):
    """
        This View takes two numbers as input and does addition of two numbers
        and provides the sum as output
    """

    def __init__(self):
        self.logger = logging.getLogger(__name__)
        self.errorObj = ErrorCodes()
        self.responseObj = ResponseJson()
        self.s_Name = "Recommend Procedure Service"
        self.status = 0
        self.patient_practice = "Adhoc"
        self.patient_id = "-1"
        self.recommended_code = {}
        extra = {
            "cls_name": self.__class__.__name__,
        }
        self.logger = logging.LoggerAdapter(self.logger, extra)

    def post(self, request):
        start_time = time.time()
        request_data = request.data
        input_request_validation_obj = InputRequestValidation()
        try:
            # req_val_res = input_request_validation_obj.request_validation(request_data)
            self.logger.info("recommendation Started")

            patient_id = request_data["Patient_ID"]
            latest_app_date = request_data["Latest_Appointment_Date"]

            patient_history = PatientData().get_patient_service_history(patient_id)
            patient_df = patient_history["response_key"]
            lab_results = PatientData().get_patient_lab_result(patient_id)
            lab_df = lab_results["response_key"]

            Patient_data, Labresult_data, resultant_df = PatientData().get_min_max_daterange(patient_df, lab_df)
            patient_df_lab_record_join, resultant_df = PatientData().get_final_df_with_lab_results(Patient_data, resultant_df)
            # print(patient_df_lab_record_join)

            one_patient_record = PatientData().get_patient_derived_data(patient_df_lab_record_join, latest_app_date)
            # print(one_patient_record['Service_Date_max'])

            if one_patient_record.loc[0]["Patient_Practice"] != "Adhoc":
                self.patient_practice = "Regular"
                recommended_code = PatientData().recommendation(one_patient_record, rules_data)
            else:
                recommended_code = PatientData().allproceducecode(rules_data)

            print(recommended_code)
            end_time = time.time()
            self.patient_id = patient_id
            self.recommended_code = recommended_code
            status_msg = self.errorObj.SuccessMsg

            self.status = 200

        except Exception as e:
            end_time = time.time()
            self.logger.exception(str(e))
            self.status = 210
            status_msg = self.errorObj.FailureMsg

        response = self.responseObj.response_json_object(
            self.s_Name + str(self.errorObj.return_error_message(str(self.status))), end_time - start_time, self.status,
            status_msg, self.patient_practice, self.patient_id, self.recommended_code
        )
        return Response(response)
